<a href="https://colab.research.google.com/github/gothiyag/firefly-intrusion-detection/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Define your GitHub repository URL
repo_url = "https://github.com/gothiyag/firefly-intrusion-detection.git"
repo_dir = "/content/firefly-intrusion-detection"

# Clone or pull the repository
if os.path.exists(repo_dir):
    # If the repo exists, pull the latest changes
    %cd {repo_dir}
    !git pull
else:
    # Clone the repository if it doesn't exist
    !git clone {repo_url}
    %cd {repo_dir}


In [ ]:
# Prompt user for Git username and email
username = input("Enter your GitHub username: ")
email = input("Enter your GitHub email: ")

# Configure Git locally for this session only
!git config user.name "{username}"
!git config user.email "{email}"

print("Git configured for this session.")